# Setup

Fall [these instructions](https://docs.astral.sh/uv/getting-started/installation/) to install `uv`. Then run the following command to set up your environment:

```bash
uv sync --quiet
```

In [1]:
# Imports
from robora import Workflow, SQLiteStorageProvider
from robora.sonar_query import SonarQueryHandler
from robora.mock_query import MockQueryHandler
from library.organization_question import OrganizationModel, get_question_set
from data import COUNTRIES, DOMAINS
import pandas as pd

In [2]:
sqlstorage = SQLiteStorageProvider("organization.db")
workflow = Workflow(
    SonarQueryHandler(OrganizationModel),
    sqlstorage,
    workers = 4,
    )
question_set = get_question_set(
    domains=['Energy'],
    countries=COUNTRIES,
)
question_set.max_questions = 0  # No limit

Loaded 3 stored question hashes from organization.db


In [3]:
await workflow.ask_multiple(question_set, return_results=False)

ask_multiple: gathering answers
ask_multiple_stream: starting for 193 questions with 4 workers
Processing question 1/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'BOLIVIA'}
Processing question 2/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'GUINEA-BISSAU'}
Processing question 3/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'NETHERLANDS'}
Processing question 4/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERGY', 'country': 'TOGO'}
Finished question 4/193
Processing question 5/193: What is the top-level state Organ (i.e., ministry/department/agency) responsible for {domain} in {country}? - {'domain': 'ENERG

[]

In [4]:
answers = []
async for answer in workflow.dump_answers():
    answers.append(answer)
flattened = pd.concat([ans.flattened for ans in answers]).reset_index(drop=True)
flattened

,question,error,domain,country,organization_name,confidence,enriched_citations
0,"What is the top-level state Organ (i.e., minis...",None,ENERGY,ALGERIA,Ministry of Energy and Renewable Energy,ConfidenceLevel.MEDIUM,[{'url': 'https://en.wikipedia.org/wiki/Minist...
1,"What is the top-level state Organ (i.e., minis...",None,ENERGY,AFGHANISTAN,Ministry of Energy and Water,ConfidenceLevel.HIGH,[{'url': 'https://en.wikipedia.org/wiki/Minist...
2,"What is the top-level state Organ (i.e., minis...",None,ENERGY,ALBANIA,Ministry of Infrastructure and Energy (Albania),ConfidenceLevel.HIGH,[{'url': 'https://www.devex.com/organizations/...
3,"What is the top-level state Organ (i.e., minis...",None,ENERGY,TOGO,Ministry of Mines and Energy,ConfidenceLevel.HIGH,[{'url': 'https://www.devex.com/organizations/...
4,"What is the top-level state Organ (i.e., minis...",None,ENERGY,GUINEA-BISSAU,Ministério da Energia e Indústria da Guiné-Bissau,ConfidenceLevel.HIGH,[{'url': 'https://energy-news-network.com/netw...
...,...,...,...,...,...,...,...
188,"What is the top-level state Organ (i.e., minis...",None,ENERGY,BHUTAN,Ministry of Energy and Natural Resources,ConfidenceLevel.HIGH,[{'url': 'https://www.developmentaid.org/organ...
189,"What is the top-level state Organ (i.e., minis...",None,ENERGY,GUINEA,"Ministry of Energy, Hydropower and Hydrocarbons",ConfidenceLevel.HIGH,[{'url': 'https://en.wikipedia.org/wiki/Minist...
190,"What is the top-level state Organ (i.e., minis...",None,ENERGY,DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA,Ministry of Atomic Energy Industry,ConfidenceLevel.HIGH,[{'url': 'https://www.shinkim.com/eng/media/ne...
191,"What is the top-level state Organ (i.e., minis...",None,ENERGY,NEPAL,"Ministry of Energy, Water Resources and Irriga...",ConfidenceLevel.HIGH,[{'url': 'https://en.wikipedia.org/wiki/Minist...


In [5]:
wrangled = flattened.copy()


In [6]:
wrangled.enriched_citations

0      [{'url': 'https://en.wikipedia.org/wiki/Minist...
1      [{'url': 'https://en.wikipedia.org/wiki/Minist...
2      [{'url': 'https://www.devex.com/organizations/...
3      [{'url': 'https://www.devex.com/organizations/...
4      [{'url': 'https://energy-news-network.com/netw...
                             ...                        
188    [{'url': 'https://www.developmentaid.org/organ...
189    [{'url': 'https://en.wikipedia.org/wiki/Minist...
190    [{'url': 'https://www.shinkim.com/eng/media/ne...
191    [{'url': 'https://en.wikipedia.org/wiki/Minist...
192    [{'url': 'https://en.wikipedia.org/wiki/Minist...
Name: enriched_citations, Length: 193, dtype: object

In [7]:
wrangled.to_excel("organization_names.xlsx", index=False)